In [ ]:
!pip install PyPDF2

In [ ]:
 pip install Unidecode

In [ ]:
from unidecode import unidecode


In [ ]:
pip install pdfplumber

*Te* amo


In [ ]:
!pip install pdfminer.six


In [ ]:
!pip install pdfminer.six unidecode


In [ ]:
!pip install pdfminer.six


In [ ]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 34.4 MB/s eta 0:00:00


In [ ]:
pip install pdfplumber

In [ ]:
import os

# Especifica el nombre de la carpeta
folder_name = "archivos_pdf"

# Crea la carpeta si no existe
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [ ]:
from google.colab import files
import os

# Crear la carpeta si no existe
folder_name = "archivos_pdf"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Cargar los archivos
uploaded_files = files.upload()

# Mover los archivos a la carpeta
for file_name, file_content in uploaded_files.items():
    file_path = os.path.join(folder_name, file_name)
    with open(file_path, "wb") as f:
        f.write(file_content)

print("Archivos cargados y guardados en la carpeta:", folder_name)


Saving 05_32_Electrónica_III.pdf to 05_32_Electrónica_III.pdf
Saving 05_33_Análisis_y_Diseño_de_Microprocesadores.pdf to 05_33_Análisis_y_Diseño_de_Microprocesadores.pdf
Saving 06_35_Comunicaciones_Analógicas.pdf to 06_35_Comunicaciones_Analógicas.pdf
Saving 06_36_Motores_y_Generadores.pdf to 06_36_Motores_y_Generadores.pdf
Saving 06_37_Diseño_Digital_con_Microcontroladores.pdf to 06_37_Diseño_Digital_con_Microcontroladores.pdf
Archivos cargados y guardados en la carpeta: archivos_pdf


In [ ]:
import os

# Especifica la carpeta donde se encuentran los archivos
folder_name = "archivos_pdf"  # Reemplaza con el nombre de tu carpeta

# Lista los archivos en la carpeta
folder_path = "/content/" + folder_name  # Ruta completa de la carpeta
file_list = os.listdir(folder_path)

# Muestra los archivos disponibles en la carpeta
print("Archivos disponibles en la carpeta:")
for i, file_name in enumerate(file_list):
    print(f"{i + 1}: {file_name}")

# Solicita al usuario que seleccione un archivo por número de índice
user_choice_index = input("Por favor, ingresa el número del archivo que deseas utilizar: ")

try:
    user_choice_index = int(user_choice_index)
    if 1 <= user_choice_index <= len(file_list):
        selected_file = file_list[user_choice_index - 1]
        pdf_path = os.path.join(folder_path, selected_file)
        print(f"Has seleccionado el archivo: {pdf_path}")
        # Ahora puedes utilizar pdf_path para trabajar con el archivo seleccionado.
    else:
        print("El número de archivo seleccionado no es válido.")
except ValueError:
    print("Por favor, ingresa un número válido.")


Archivos disponibles en la carpeta:
1: 05_32_Electrónica_III.pdf
2: 05_33_Análisis_y_Diseño_de_Microprocesadores.pdf
3: 06_37_Diseño_Digital_con_Microcontroladores.pdf
4: 06_35_Comunicaciones_Analógicas.pdf
5: 06_36_Motores_y_Generadores.pdf
Por favor, ingresa el número del archivo que deseas utilizar: 2
Has seleccionado el archivo: /content/archivos_pdf/05_33_Análisis_y_Diseño_de_Microprocesadores.pdf


In [ ]:
import pdfplumber
import re
from unidecode import unidecode

def extract_section_with_title(pdf_path):
    section_content = []
    capture_content = False

    # Palabras clave a buscar (cualquiera de ellas)
    search_keywords = ["PROGRAMA SINTÉTICO","PROGRAMA SINTETICO", "Programa Sintetico", "ROGRAMA SINTETICO","PROGRAMA DETALLADO", "Temas","UNIDADES","Unidades tematicas"]

    roman_pattern = re.compile(r'^(I{1,3}|IV|V?I{0,3}|IX|X{1,3})\.\s+')
    digits_pattern = re.compile(r'^\d')


    with pdfplumber.open(pdf_path) as pdf:
        text_content = []
        for page in pdf.pages:
            text = page.extract_text()
            text_content.extend(text.split('\n'))

        current_line = ""
        grouping_by_digit = False
        for i, line in enumerate(text_content):
            line_without_accent = unidecode(line).strip()

            # Comenzar a capturar si encuentra cualquiera de las palabras clave en la línea
            if any(keyword in line_without_accent for keyword in search_keywords):
                capture_content = True

            # Detener la captura si encontramos cualquiera de las palabras clave de detención
            if "Resultados de Aprendizaje" in line_without_accent or "PROGRAMA POR UNIDADES" in line_without_accent or "Evaluacion" in line_without_accent or "RESULTADOS" in line_without_accent or "Metodologia" in line_without_accent  or "ESTRATEGIAS" in line_without_accent or "METODOLOGIA" in line_without_accent or "PEDAGOGICA" in line_without_accent or "EVALUACION" in line_without_accent:
                capture_content = False
            elif line_without_accent.isupper() and text_content.count(line) > 1:
                capture_content = False

            if capture_content:
                if roman_pattern.match(line_without_accent):
                    if current_line:
                        section_content.append(current_line)
                        current_line = ""
                    if i + 1 < len(text_content) and digits_pattern.match(unidecode(text_content[i + 1]).strip()):
                        grouping_by_digit = True
                        current_line = line_without_accent
                    else:
                        section_content.append(line_without_accent)
                elif grouping_by_digit:
                    current_line += " " + line_without_accent
                    if digits_pattern.match(line_without_accent) and "." in line_without_accent:
                        section_content.append(current_line)
                        current_line = ""
                        grouping_by_digit = False
                else:
                    section_content.append(line_without_accent)

    for i, line in enumerate(section_content):
        # Eliminar las ocurrencias de 'x' o 'X' que estén rodeadas de espacios o en el inicio/final de una línea
        cleaned_line = re.sub(r'\s+[xX]\s+|(?<=^)[xX]\s+|\s+[xX](?=$)', ' ', line)
        # Eliminar la palabra "pagina"
        cleaned_line = re.sub(r'\bpagina\b', '', cleaned_line, flags=re.IGNORECASE)
        # Eliminar cualquier 'x' aislada que aún esté presente, sin dejar múltiples espacios
        cleaned_line = re.sub(r'\s*x\s*', ' ', cleaned_line, flags=re.IGNORECASE)
        section_content[i] = cleaned_line.strip()  # Añadir .strip() para eliminar posibles espacios adicionales al principio o al final

    return section_content

# Para utilizar la función con la variable pdf_path ya definida
section = extract_section_with_title(pdf_path)
for line in section:
    print(line)





PROGRAMA SINTETICO
Lenguajes de descripcion de hardware
Diseno, sintesis y optimizacion de maquinas de estado algoritmicas.
Analisis y programacion de procesadores CISC y RISC.
